In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import patches
from matplotlib.patches import Ellipse
from decimal import Decimal

In [ ]:
plt.rcParams['font.size'] = 7

In [ ]:
target = '~/Downloads/MSName_RandChunk0_DynSpecs_1556208112/TARGET/1556208112_14:29:42.950_-62:40:46.200.fits'
target_w = '~/Downloads/MSName_RandChunk0_DynSpecs_1556208112/TARGET_W/1556208112_14:29:42.950_-62:40:46.200.W.fits'

target_hdu = fits.open(target)[0]
target_w_hdu = fits.open(target_w)[0]

data = target_hdu.data
weights = target_w_hdu.data
# print(target_hdu.shape)
# print(target_w_hdu.shape)
# print(target_hdu.header)

In [ ]:
norm_w = np.sqrt(weights)
for i in range(4):
    data[i, :, :] = data[i, :, :] * norm_w * np.cbrt(data[i, :, :])

# Stokes I, Q, U, V Plotted in order below
```python
I = data[0, :, :]
Q = data[1, :, :]
U = data[2, :, :]
V = data[3, :, :]
```

In [ ]:
def plot_stokes(d, s, f=15, t0=6325, t1=6345):
    """Plot dynamic spectra for given data array and Stokes param
    
    Parameters
    ----------
    d: array
        The data array extracted from fits file
    s: int
        Stokes parameter of interest mapped (I, Q, U, V)->(0, 1, 2, 3)
    f: int
        channel slice
    t0: int
        TBD - will be the `start_time` to zoom-in on
    t1: int
        TBD - will be the `end_time to` zoom-in on
    """
    # Slice in time at channel 15 arbitrarily
    print("--- Stats ---")
    print("Min: ", np.min(d[s, :, t0:t1]))
    print("Max: ", np.max(d[s, :, t0:t1]))
    print("Noise: ", np.std(d[s, :, t0:t1]))
    print("Mean: ", np.mean(d[s, :, t0:t1]))
    print("--- Time for max flux at this frequency ---")
    print(np.argmax(d[s, f, :]))
    print("-------------------------------------------")
    data_sl = d[s, :, t0:t1]

    # Plot
    plt.imshow(data_sl, origin='lower', cmap='viridis', aspect='auto', vmin=np.min(data_sl), vmax=np.max(data_sl))
    cbar = plt.colorbar(pad=.07)
    cbar.set_label(r'$Jy/B$', size=7)

In [ ]:
plot_stokes(data, 0)

In [ ]:
plot_stokes(data, 1)

In [ ]:
plot_stokes(data, 2)

In [ ]:
plot_stokes(data, 3)